In [1]:
import pandas as pd

In [5]:
pd.set_option('display.max_rows', 10000)

# Swiss population and naturalization data - Communes
* In this section, we take the BFS-data on the Swiss population, which is stored in excel format.
* We need to open the file and go through the sheets (2015, 2014, etc).
* Watch out because the layout of the excel sheet keeps changing
* We also need to separate the rows that contain canton, district and communal data.
* Everything goes into a long table, which we will turn into wide format (one row for each year later on).

In [109]:
#This will get us set up to read the various sheets in the excel-file su-d-01.02.04.08.xls
def read_populationdata(year):

    colnames_1991 = ['Geografische Einheit', 'Gemeindename', 'Bevölkerungsstand 1. Jan', 'Lebendgeburten', 'Todesfälle', 'Geburtenüberschuss', 'Zuzüge', 'Wegzüge', 'Wanderungssaldo', 'Einbürgerungen', 'Bereinigungen', 'Bevölkerungsstand 31. Dez', 'Veränderung absolut', 'Veränderung relativ']
    colnames_2006 = ['Geografische Einheit', 'Bevölkerungsstand 1. Jan', 'Lebendgeburten', 'Todesfälle', 'Geburtenüberschuss', 'Zuzüge', 'Wegzüge', 'Wanderungssaldo', 'Einbürgerungen', 'Bereinigungen', 'Bevölkerungsstand 31. Dez', 'Veränderung absolut', 'Veränderung relativ']
    
    if year in range(1991, 2005):
        df = pd.read_excel('OriginalData/su-d-01.02.04.08.xls', sheetname=str(year), skiprows=4, names=colnames_1991)
        df['GemeindeID'] = df['Geografische Einheit'].astype(str).str.extract('^(\d+)$')
    
    #in 2005, the contents of Column B were placed in Column A
    elif year in range(2005, 2006):
        df = pd.read_excel('OriginalData/su-d-01.02.04.08.xls', sheetname=str(year), skiprows=4, names=colnames_1991)
        df['GemeindeID'] = df['Geografische Einheit'].astype(str).str.extract('^\s*(\d+) ')
        df['Gemeindename'] = df['Geografische Einheit'].astype(str).str.extract('^\s*\d+ (.*)')
    
    #starting in 2006, they deleted Column B in the excel
    elif year in range(2006, 2008):
        df = pd.read_excel('OriginalData/su-d-01.02.04.08.xls', sheetname=str(year), skiprows=4, names=colnames_2006)
        df['GemeindeID'] = df['Geografische Einheit'].astype(str).str.extract('^\s*(\d+) ')
        df['Gemeindename'] = df['Geografische Einheit'].astype(str).str.extract('^\s*\d+ (.*)')
    
    #starting in 2008, they put the first data row in column 6, not 5    
    elif year in range(2008, 2011):
        df = pd.read_excel('OriginalData/su-d-01.02.04.08.xls', sheetname=str(year), skiprows=5, names=colnames_2006)
        df['GemeindeID'] = df['Geografische Einheit'].astype(str).str.extract('^\s*(\d+) ')
        df['Gemeindename'] = df['Geografische Einheit'].astype(str).str.extract('^\s*\d+ (.*)')
    
    #starting in 2011, they added funny characters to Column A    
    else:
        df = pd.read_excel('OriginalData/su-d-01.02.04.08.xls', sheetname=str(year), skiprows=5, names=colnames_2006)
        df['GemeindeID'] = df['Geografische Einheit'].astype(str).str.extract('^\.{6}(\d{4})')
        df['Gemeindename'] = df['Geografische Einheit'].astype(str).str.extract('^\.{6}\d{4} (.*)')

    #Get rid of all rows that don't represent a commune
    df = df[df['GemeindeID'].notnull()]
    
    #At this point we can convert everything in GemeindeID into an int
    df['GemeindeID'] = df['GemeindeID'].astype(int)
    
    #Add a year column to the dataframe
    df['Jahr'] = year
    
    #We can get rid of most of the columns at this point
    df.pop('Geografische Einheit')
    df.pop('Lebendgeburten')
    df.pop('Todesfälle')
    df.pop('Geburtenüberschuss')
    df.pop('Zuzüge')
    df.pop('Wegzüge')
    df.pop('Wanderungssaldo')
    df.pop('Bereinigungen')
    df.pop('Bevölkerungsstand 31. Dez')
    df.pop('Veränderung absolut')
    df.pop('Veränderung relativ')
    
    return df

In [110]:
#Testing
year = 2010
df = read_populationdata(year)
df

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:26: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:27: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)


,Bevölkerungsstand 1. Jan,Einbürgerungen,GemeindeID,Gemeindename,Jahr
3,1538.0,4.0,1,Aeugst am Albis,2010
4,8022.0,106.0,2,Affoltern am Albis,2010
5,4460.0,15.0,3,Bonstetten,2010
6,2934.0,10.0,4,Hausen am Albis,2010
7,2950.0,37.0,5,Hedingen,2010
8,788.0,2.0,6,Kappel am Albis,2010
9,1552.0,2.0,7,Knonau,2010
10,550.0,0.0,8,Maschwanden,2010
11,3679.0,10.0,9,Mettmenstetten,2010
12,3816.0,23.0,10,Obfelden,2010


In [111]:
#We now call the function for all years in question and store the dfs in a list
dfs = []
for year in range(1991, 2016):
    df = read_populationdata(year)
    dfs.append(df)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:9: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  if __name__ == '__main__':
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:14: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:15: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:20: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a futu

In [112]:
#We cram all the 25 dfs together to one df (with 60k rows)
df = pd.concat(dfs)

In [113]:
#At this point, we save it into a csv
df.to_csv('CleanData/Gem-EinbBev-Long-1991-2995.csv')

In [116]:
df[df['GemeindeID'] == 951]

,Bevölkerungsstand 1. Jan,Einbürgerungen,GemeindeID,Gemeindename,Jahr
573,1137.0,0.0,951,Affoltern im Emmental,1991
573,1156.0,0.0,951,Affoltern im Emmental,1992
573,1182.0,0.0,951,Affoltern im Emmental,1993
573,1187.0,1.0,951,Affoltern im Emmental,1994
573,1189.0,0.0,951,Affoltern im Emmental,1995
573,1191.0,0.0,951,Affoltern im Emmental,1996
573,1159.0,0.0,951,Affoltern im Emmental,1997
573,1160.0,0.0,951,Affoltern im Emmental,1998
573,1178.0,0.0,951,Affoltern im Emmental,1999
573,1159.0,0.0,951,Affoltern im Emmental,2000


In [118]:
#We create a pivot table for Einbürgerungen (naturalizations)
#BUT WAIT WITH THIS UNTIL WE HAVE FIXED THE ISSUE WITH CHANGING GEMEINDE-IDS!!!!!!!!!!!!!!!!
df_gem = df.pivot(index='GemeindeID', columns='Jahr', values='Einbürgerungen').reset_index()

In [125]:
df_gem.to_csv('CleanData/Gem-Einb-Wide-1991-2015.csv', index=False)

In [122]:
#Same procedure with Bevölkerung (population)
df_bev = df.pivot(index='GemeindeID', columns='Jahr', values='Bevölkerungsstand 1. Jan').reset_index()

In [124]:
df_bev.to_csv('CleanData/Gem-Bev-Wide-1991-2015.csv', index=False)

Additional code to create a reference table for Communes and their IDs

In [90]:
#Using the year 2015, we also create a list of commune IDs with their names
df_2015 = read_populationdata(2015)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:32: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:33: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)


In [92]:
df_2015.pop('Bevölkerungsstand 1. Jan')
df_2015.pop('Einbürgerungen')
df_2015.pop('Jahr')

3       2015
4       2015
5       2015
6       2015
7       2015
8       2015
9       2015
10      2015
11      2015
12      2015
13      2015
14      2015
15      2015
16      2015
18      2015
19      2015
20      2015
21      2015
22      2015
23      2015
24      2015
25      2015
26      2015
27      2015
28      2015
29      2015
30      2015
31      2015
32      2015
33      2015
34      2015
35      2015
36      2015
37      2015
38      2015
39      2015
40      2015
41      2015
43      2015
44      2015
45      2015
46      2015
47      2015
48      2015
49      2015
50      2015
51      2015
52      2015
53      2015
54      2015
55      2015
56      2015
57      2015
58      2015
59      2015
60      2015
61      2015
62      2015
63      2015
64      2015
66      2015
67      2015
68      2015
69      2015
70      2015
71      2015
72      2015
73      2015
74      2015
75      2015
76      2015
77      2015
78      2015
79      2015
80      2015
81      2015
82      2015

In [94]:
#We save the table for 2015 for reference
df_2015.to_csv('CleanData/GemIDName-2015.csv')